In [1]:
# !pip install xgboost==0.90

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gc
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import pickle
import os

In [3]:
data = pd.read_pickle('./data/data_all.pickle')

In [5]:
def train_test_split(data):
    origin_feature   = ['用户二分类特征A', '用户二分类特征B', '用户二分类特征C','用户二分类特征D', '用户二分类特征E','用户的盐值分数']
    lbl_enc_feat = ['性别_lbl_enc','访问频率_lbl_enc','用户分类特征A_lbl_enc', '用户分类特征B_lbl_enc', '用户分类特征C_lbl_enc', '用户分类特征D_lbl_enc','用户分类特征E_lbl_enc']
    user_ques_svd    = ['问题绑定的话题ID_svd_{}'.format(i) for i in range(1,31)]
    user_ques_watched_svd  = ['用户关注的话题_svd_{}'.format(i) for i in range(1,11)]
    user_ques_fav_svd  = ['user_ques_fav_svd_{}'.format(i) for i in range(1,11)]
    nn_stack = ['w2v_sum_nn']

    test_feat = ['问题邀请用户_counts','用户被邀请问题_counts','用户的盐值分数_max',
                 '用户的盐值分数_min','用户的盐值分数_std','用户的盐值分数_mean','invite_answer_gap','邀请问题创建时间_gap',
                '用户关注的话题_len','用户感兴趣的话题_len','问题绑定的话题ID_len','prev_ans_ques_title_sim_min', 'prev_ans_ques_title_sim_max',
           'prev_ans_ques_title_sim_mean', 'prev_ans_ques_title_sim_std']
    
    new_feat = ['用户ID_last_expo','用户ID_next_expo','问题ID_last_expo','问题ID_next_expo','问题创建时间_H','邀请创建时间_H']
    
    new_feat_II = ['用户_问题IDs_svd_{}'.format(i) for i in range(1,11)]+['问题_用户IDs_svd_{}'.format(i) for i in range(1,11)]+['lstm_enc_feat']+\
     ['邀请创建时间_mean','邀请创建时间_max','邀请创建时间_min','邀请创建时间_std','问题创建时间_mean','问题创建时间_max','问题创建时间_min',
      '问题创建时间_std','用户ID曾经回答数','用户问题ID_count','用户问题话题相同个数','用户感兴趣问题话题相同个数']
      
    new_feat_III = ['问题_用户感兴趣_topic_sim','问题_用户关注_topic_sim','用户关注_感兴趣_topic_sim']
    
    new_feat_IV = ['回答是否被标优_count', '回答是否被推荐_count' , '是否包含图片_count' ,
                   '是否包含视频_count','回答字数_mean' ,'点赞数_mean', '取赞数_mean' ,'评论数_mean' ,
                   '收藏数_mean', '感谢数_mean' ,'举报数_mean', '没有帮助数_mean' ,'反对数_mean']
    
    new_feat_V = ['回答字数_sum' ,'点赞数_sum', '取赞数_sum' ,'评论数_sum' ,'收藏数_sum', '感谢数_sum' ,
                  '举报数_sum', '没有帮助数_sum' ,'反对数_sum']

    new_feat_VI = ['问题标题_曾经回答_SW_sim','问题描述_曾经回答_SW_sim','问题标题_曾经回答_W_sim','问题描述_曾经回答_W_sim','times_mean',
                   'times_max','times_min','times_std','Hs_mean','Hs_max','Hs_min','Hs_std']
    
    new_feat_VII = ['问题描述_曾经回答_Topic_sim','prev_topic_sims_min','prev_topic_sims_max','prev_topic_sims_mean','prev_topic_sims_std']
    feat_I =['question_curr_expo','user_curr_expo']
    feat_II = ['prev_ans_times_min_gap', 'prev_ans_times_mean_gap', 'qtime_std','qtime_mean', 'utime_std', 'utime_mean', 'iweek', 'qlast_itime_gap','qllast_itime_gap', 'qlllast_itime_gap', 'qnext_itime_gap','qnnext_itime_gap', 'qnnnext_itime_gap', 'ulast_itime_gap','ullast_itime_gap', 'ulllast_itime_gap', 'unext_itime_gap','unnext_itime_gap', 'unnnext_itime_gap']
    new_feat_VIII = feat_I + feat_II 
    ques_len_stat_feat = ['问题标题_len', '问题描述_len', '问题描述_len-问题标题_len', '问题标题_W_len', '问题描述_W_len','问题描述_len-问题标题_len_W', '问题标题和描述的交集个数', '问题标题和描述的交集个数_W', '问题标题和描述的交集个数/问题标题_len', '问题标题和描述的交集个数/问题描述_len', '问题标题和描述的交集个数_W/问题标题_len', '问题标题和描述的交集个数_W/问题描述_len', '编辑距离',
       '前一个词语是否相同', '前两个词语是否相同', '前三个词语是否相同', '前一个词语是否相同_W', '前两个词语是否相同_W','前三个词语是否相同_W','第一个词语在标题里面出现位置', '第二个词语在标题里面出现位置', '第三个词语在标题里面出现位置','第一感兴趣在问题绑定话题里面出现位置', '第二感兴趣在问题绑定话题里面出现位置', '第三感兴趣在问题绑定话题里面出现位置', '问题话题编辑距离',
       '问题标题_len_mean', '问题描述_len_mean', '问题标题_W_len_mean', '问题描述_W_len_mean']
    #******* Feature sum***********#
    feature  =    origin_feature + lbl_enc_feat + user_ques_svd + user_ques_watched_svd + user_ques_fav_svd + nn_stack +\
                   test_feat + new_feat + new_feat_II + new_feat_III + new_feat_IV + new_feat_V + new_feat_VI+\
                  new_feat_VII + new_feat_VIII + ques_len_stat_feat
    
    #*********************************# 
    len_train = 9489162
#     train = data[:len_train]
#     test = data[len_train:]
#     test_index = np.isnan(data['邀请是否被回答'])
#     train_index = ~test_index
    train_x = data[:len_train][feature] 
    train_y = data[:len_train]['邀请是否被回答']
    test_x  = data[len_train:][feature]
    train_x.columns = ['col_{}'.format(i) for i in range(211)]
    test_x.columns = ['col_{}'.format(i) for i in range(211)]
    return train_x, train_y, test_x,feature
    
train_x, train_y, test_x,feature = train_test_split(data)
del data
gc.collect()
print('All features: train shape {}, test shape {}'.format(train_x.shape, test_x.shape))
print(len(feature))

All features: train shape (9489162, 211), test shape (1141718, 211)
211


In [6]:
tic = time.time()
preds = np.zeros((test_x.shape[0], 2))
scores = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('*' * 30)
    X_train, y_train, X_valid, y_valid = train_x.iloc[tr_idx], train_y.iloc[tr_idx], train_x.iloc[va_idx], train_y.iloc[va_idx]

    xgb_model = xgb.XGBClassifier(
                                n_estimators=50000,
                                max_depth=12,
                                learning_rate=0.1,
                                reg_lambda=10,
                                subsample=0.8,
                                colsample_bytree=0.8,
                                missing=np.nan,
                                random_state=42,
                                tree_method='gpu_hist'  
                            )
    xgb_model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], eval_metric='auc', early_stopping_rounds=300, verbose=100)

    score = xgb_model.best_score
    scores.append(score)
    print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, xgb_model.best_iteration, score,np.mean(scores))) 
    pred = xgb_model.predict_proba(test_x)
    preds += pred  #mean score 0.8970
    np.save('./out/xgb_preds_{}.npy'.format(index+1), pred)

    pickle.dump(xgb_model, open('./models/main_xgb_f%d.pkl' % index, "wb"))

    del xgb_model
    del X_train, y_train, X_valid, y_valid
    gc.collect()
    
toc = time.time()
print("Wall time: %d s" % (toc-tic))

******************************
[0]	validation_0-auc:0.808866
Will train until validation_0-auc hasn't improved in 300 rounds.
[100]	validation_0-auc:0.889244
[200]	validation_0-auc:0.89408
[300]	validation_0-auc:0.895997
[400]	validation_0-auc:0.896953
[500]	validation_0-auc:0.897522
[600]	validation_0-auc:0.897778
[700]	validation_0-auc:0.89799
[800]	validation_0-auc:0.898137
[900]	validation_0-auc:0.898193
[1000]	validation_0-auc:0.898212
[1100]	validation_0-auc:0.898234
[1200]	validation_0-auc:0.898238
[1300]	validation_0-auc:0.898224
[1400]	validation_0-auc:0.898172
[1500]	validation_0-auc:0.898144
Stopping. Best iteration:
[1245]	validation_0-auc:0.898258

fold 1 round 1245 : score: 0.898258 | mean score 0.898258
******************************
[0]	validation_0-auc:0.808905
Will train until validation_0-auc hasn't improved in 300 rounds.
[100]	validation_0-auc:0.888921
[200]	validation_0-auc:0.894238
[300]	validation_0-auc:0.896161
[400]	validation_0-auc:0.89701
[500]	validation_0-

In [7]:
invite_info_evaluate = pd.read_table('../data/data_set_0926/invite_info_evaluate_2_0926.txt',header=None)
invite_info_evaluate.columns =  ['问题ID','用户ID','邀请创建时间']
result = invite_info_evaluate[['问题ID','用户ID','邀请创建时间']]
result['result'] = preds[:, 1] / 5
result.to_csv('./out/result_xgb_nouser.txt', sep='\t', index=False, header=False)